Connection to API

In [1]:
import http.client
import json

conn = http.client.HTTPSConnection("google-flights2.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "ef81eb96e9msh5c4170aadaa7789p118793jsnca6aae8d2439",
    'x-rapidapi-host': "google-flights2.p.rapidapi.com"
}

Fetching the airports data:

In [2]:

conn.request("GET", f"/api/v1/searchAirport?query=Italy&language_code=en-US", headers=headers)

res = conn.getresponse()
data = res.read()
data = data.decode("utf-8")
response = json.loads(data)

if response.get("status") and response.get("message") == "Success":
    country_data = {
        "country": "Italy",
        "cities": {}
    }

    for city_data in response.get("data", []):
        city_name = city_data["city"]

        airports = [
            {"id": item["id"], "name": item["title"]}
            for item in city_data.get("list", [])
            if item["type"] == "airport"
        ]

        if airports:
            country_data["cities"][city_name] = airports

else:
    print(f"API response indicates failure for country: Italy")

print(json.dumps(country_data, indent=2))

{
  "country": "Italy",
  "cities": {
    "Rome": [
      {
        "id": "FCO",
        "name": "Leonardo da Vinci International Airport"
      },
      {
        "id": "CIA",
        "name": "Giovan Battista Pastine International Airport"
      }
    ],
    "Florence": [
      {
        "id": "FLR",
        "name": "Amerigo Vespucci Airport"
      },
      {
        "id": "PSA",
        "name": "Pisa International Airport"
      }
    ],
    "Venice": [
      {
        "id": "VCE",
        "name": "Venice Marco Polo Airport"
      },
      {
        "id": "TSF",
        "name": "Treviso Airport"
      }
    ],
    "Milan": [
      {
        "id": "MXP",
        "name": "Milano Malpensa Airport"
      },
      {
        "id": "BGY",
        "name": "Il Caravaggio International Airport"
      },
      {
        "id": "LIN",
        "name": "Milan Linate Airport"
      }
    ],
    "Naples": [
      {
        "id": "NAP",
        "name": "Naples International Airport"
      }
    ]
  }


Getting the flight prices from API 

In [4]:
departure_id = "FCO"
arrival_id = "MXP"
outbound_date = "2025-04-01"
conn.request(
    "GET",
    f"/api/v1/searchFlights?departure_id={departure_id}&arrival_id={arrival_id}&outbound_date={outbound_date}"
    "&travel_class=ECONOMY&adults=1&show_hidden=1&currency=EUR&language_code=en-US",
    headers=headers
)

try:
    res = conn.getresponse()
    if res.status != 200:
        print(f"Failed to fetch flight data: HTTP {res.status} {res.reason}")
        conn.close()
        exit()

    data = res.read().decode("utf-8")
    response = json.loads(data)

    if response.get("status") and response.get("message") == "Success":
        itineraries = response.get("data", {}).get("itineraries", {})
        top_flights = itineraries.get("topFlights", [])
        if top_flights:
            top_two_flights = top_flights[:2]
            print("Top 2 Flights:")
            for i, flight in enumerate(top_two_flights, start=1):
                print(f"Flight {i}:")
                print(json.dumps(flight, indent=2))
        else:
            print("No top flights found.")
    else:
        print(f"API response indicates failure: {response.get('message', 'Unknown error')}")
except Exception as e:
    print(f"An error occurred: {str(e)}")
finally:
    conn.close()

Top 2 Flights:
Flight 1:
{
  "departure_time": "01-04-2025 09:30 AM",
  "arrival_time": "01-04-2025 10:55 AM",
  "duration": {
    "raw": 85,
    "text": "1 hr 25 min"
  },
  "flights": [
    {
      "departure_airport": {
        "airport_name": "Leonardo da Vinci International Airport",
        "airport_code": "FCO",
        "time": "2025-4-1 09:30"
      },
      "arrival_airport": {
        "airport_name": "Milano Malpensa Airport",
        "airport_code": "MXP",
        "time": "2025-4-1 10:55"
      },
      "duration": {
        "raw": 85,
        "text": "1 hr 25 min"
      },
      "airline": "AEROITALIA SRL",
      "airline_logo": "https://www.gstatic.com/flights/airline_logos/70px/XZ.png",
      "flight_number": "XZ 2021",
      "aircraft": "ATR 72",
      "seat": "Average legroom",
      "legroom": "30 in",
      "extensions": [
        "Average legroom (30 in)",
        "Emissions estimate: 58 kg CO2e"
      ]
    }
  ],
  "delay": {
    "values": false,
    "text": 0
  },